In [21]:
import os
import pandas as pd
import pathlib
import re
from glob import glob
import xml.etree.ElementTree as ET
import spacy 
from spacy import displacy

In [11]:
#!pip install git+git://github.com/petermr/pygetpapers
QUERY = "ethics statement frontiers"
HITS = 100
OUTPUT = 'ethics_statement_frontiers_100'
!pygetpapers -q "$QUERY" -k $HITS -o $OUTPUT -x


INFO: Total Hits are 75146
INFO: Saving XML files to C:\Users\shweata\dictionary\ethics_statement_project\ethics_statement_frontiers_100\*\fulltext.xml
INFO: */Wrote xml for PMC8177047/
INFO: */Wrote xml for PMC8160237/
INFO: */Wrote xml for PMC8138029/
INFO: */Wrote xml for PMC8177669/
INFO: */Wrote xml for PMC8172783/
INFO: */Wrote xml for PMC8177695/
INFO: */Wrote xml for PMC8129497/
INFO: */Wrote xml for PMC8023327/
INFO: */Wrote xml for PMC8144287/
INFO: */Wrote xml for PMC8185063/
INFO: */Wrote xml for PMC8183679/
INFO: */Wrote xml for PMC8134681/
INFO: */Wrote xml for PMC8173221/
INFO: */Wrote xml for PMC8129560/
INFO: */Wrote xml for PMC8185018/
INFO: */Wrote xml for PMC8176203/
INFO: */Wrote xml for PMC8027102/
INFO: */Wrote xml for PMC8149743/
INFO: */Wrote xml for PMC7959173/
INFO: */Wrote xml for PMC8144523/
INFO: */Wrote xml for PMC8110698/
INFO: */Wrote xml for PMC8138428/
INFO: */Wrote xml for PMC8119626/
INFO: */Wrote xml for PMC8180579/
INFO: */Wrote xml for PMC8116530

In [12]:
#!git clone https://github.com/petermr/ami3.git
#!cd ami3
#!mvn install -Dmaven.test.skip=true
!ami -p $OUTPUT section


Generic values (AMISectionTool)

Specific values (AMISectionTool)
AMISectionTool cTree: PMC7906226
AMISectionTool cTree: PMC7940375
AMISectionTool cTree: PMC7947343
AMISectionTool cTree: PMC7959173
AMISectionTool cTree: PMC7969973
AMISectionTool cTree: PMC7982648
AMISectionTool cTree: PMC7987664
AMISectionTool cTree: PMC7994513
AMISectionTool cTree: PMC8005519
AMISectionTool cTree: PMC8011658
AMISectionTool cTree: PMC8012510
AMISectionTool cTree: PMC8017196
AMISectionTool cTree: PMC8019914
AMISectionTool cTree: PMC8020902
AMISectionTool cTree: PMC8023327
AMISectionTool cTree: PMC8027102
AMISectionTool cTree: PMC8027481
AMISectionTool cTree: PMC8032858
AMISectionTool cTree: PMC8033151
AMISectionTool cTree: PMC8042130
AMISectionTool cTree: PMC8044422
AMISectionTool cTree: PMC8044765
AMISectionTool cTree: PMC8060433
AMISectionTool cTree: PMC8060638
AMISectionTool cTree: PMC8062708
AMISectionTool cTree: PMC8062769
AMISectionTool cTree: PMC8076538
AMISectionTool cTree: PMC8079727
AMISectio

In [17]:
HOME= 'C:/Users/shweata/dictionary/ethics_statement_project'
ethics_statements = glob(os.path.join(HOME, OUTPUT, 'PMC*', 'sections','*', '[0-9]_ethic*', '[1_9]_p.xml'))
ethics_statements
#len(ethics_statements)

96

In [19]:
file1 = open("ethics_statements_frontiers_100.txt","w+", encoding='utf-8')
for file in ethics_statements:
    tree = ET.parse(file)
    root = tree.getroot()
    for para in root.iter('p'):
        #file1.write('para.text')
        print (para.text, file = file1)

text = pathlib.Path("ethics_statements_frontiers_100.txt").read_text(encoding='utf-8')

In [29]:
nlp = spacy.load("en_core_web_sm")


#!python -m pip install icecream
#from icecream import ic
doc = nlp(text)


entities = []
labels = []
position_start = []
position_end = []

for ent in doc.ents:
    entities.append(ent)
    labels.append(ent.label_)
    position_start.append(ent.start_char)
    position_end.append(ent.end_char)
    
df = pd.DataFrame({'Entities':entities,'Labels':labels,'Position_Start':position_start, 'Position_End':position_end})
#pd.set_option("display.max_rows", None, "display.max_columns", None)
df.to_csv('entity_recognition_spacy_ethics_statement_frontiers_100.csv', encoding='utf-8')
#ic(entities, labels)


In [25]:
pattern = re.compile(r'approved\sby\sthe\s[a-zA-Z]*.')
matches=pattern.finditer(text)
for match in matches:
    print(match)

<re.Match object; span=(856, 879), match='approved by the Ethics '>
<re.Match object; span=(1747, 1777), match='approved by the Institutional '>
<re.Match object; span=(2164, 2187), match='approved by the ethics '>
<re.Match object; span=(3300, 3324), match='approved by the Medical '>
<re.Match object; span=(3631, 3654), match='approved by the Ethics '>
<re.Match object; span=(3911, 3934), match='approved by the ethics '>
<re.Match object; span=(4156, 4179), match='approved by the ethics '>
<re.Match object; span=(6080, 6104), match='approved by the Medical '>
<re.Match object; span=(6496, 6519), match='approved by the Ethics '>
<re.Match object; span=(7501, 7524), match='approved by the ethics '>
<re.Match object; span=(8437, 8460), match='approved by the Ethics '>
<re.Match object; span=(9844, 9874), match='approved by the institutional '>
<re.Match object; span=(10127, 10151), match='approved by the Medical '>
<re.Match object; span=(13009, 13033), match='approved by the Medical '>


In [35]:
pattern = re.compile(r'not\srequired\s[a-zA-Z]*.')
matches=pattern.findall(text)
for match in matches:
    print(match)
len(matches)

not required for 
not required for 
not required to 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 
not required for 


21

In [38]:
#!pip install sparqlwrapper

import sys
from SPARQLWrapper import SPARQLWrapper, JSON, XML
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11")



In [36]:
endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT DISTINCT ?org ?orgLabel ?acronym WHERE {
  ?org wdt:P3500 ?ringgold .
  ?org wdt:P1813 ?acronym .
  ?person wdt:P108 ?org .
  ?paper wdt:P50 ?person .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    
    sparql.setReturnFormat(XML)
    return sparql.query().convert()

results_xml = get_results(endpoint_url, query)

for result in results_xml["results"]["bindings"]:
    print(result)


TypeError: 'Document' object is not subscriptable

In [39]:
import sys
from SPARQLWrapper import SPARQLWrapper, XML

endpoint_url = "https://query.wikidata.org/sparql"

query = """#research council
SELECT ?researchcouncil ?researchcouncilLabel 
WHERE 
{
  ?researchcouncil wdt:P31 wd:Q10498148.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(XML)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


TypeError: 'Document' object is not subscriptable

[SparqlKernelInstall] Installing SPARQL kernel
[SparqlKernelInstall] Installing kernel spec
[SparqlKernelInstall] Installed kernelspec sparql in C:\ProgramData\jupyter\kernels\sparql
[SparqlKernelInstall] Installed into C:\ProgramData\jupyter\kernels\sparql
[SparqlKernelInstall] Installing CSS
[SparqlKernelInstall] Installing CSS into c:\users\shweata\appdata\local\programs\python\python39\lib\site-packages\notebook\static\custom
Traceback (most recent call last):
  File "c:\users\shweata\appdata\local\programs\python\python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\shweata\appdata\local\programs\python\python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\shweata\AppData\Local\Programs\Python\Python39\Scripts\jupyter-sparqlkernel.EXE\__main__.py", line 7, in <module>
    sys.exit(main())
  File "c:\users\shweata\appdata\local\programs\python\python39\lib\site-packages\sparqlker